In [ ]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Carbon Fiber
CF_E1 = joblib.load("CF_E1")
CF_E2 = joblib.load("CF_E2")
CF_Poissons_ratio_12=joblib.load("CF_Poissons_ratio_12")
CF_Shear_Modulus_12=joblib.load("CF_Shear_Modulus_12")
CF_Shear_Modulus_13=joblib.load("CF_Shear_Modulus_13")
CF_Shear_Modulus_23=joblib.load("CF_Shear_Modulus_23")
CF_Area_Density=joblib.load("CF_Area_Density")
CF_Tensile_Strength=joblib.load("Tensile Strength")

In [ ]:
#Input Parameters by User
#Call Data Frame
df=pd.read_excel("Input Parameters3.xlsx")
labelencoder=LabelEncoder()
df['Weave']=labelencoder.fit_transform(df['Weave'])

#Separate Fiber Types, Carbon Fiber
df_cf=df[(df['Type']=="Carbon")]

In [ ]:
df

In [ ]:
list_of_material = df['Type'].tolist()
list_of_material

In [ ]:
# Predict Mechanical Properties
#Carbon Fiber
cf=df_cf[["Warp_Count","Weft_Count","Thickness","Warp_Filament_Count","Weft_Filament_Count","Weave"]]
df_cf['E1']=CF_E1.predict(cf)
df_cf['E2']=CF_E2.predict(cf)
df_cf['Poissons_ratio_12']=CF_Poissons_ratio_12.predict(cf)
df_cf['Shear_Modulus_12']=CF_Shear_Modulus_12.predict(cf)
df_cf['Shear_Modulus_13']=CF_Shear_Modulus_13.predict(cf)
df_cf['Shear_Modulus_23']=CF_Shear_Modulus_23.predict(cf)
df_cf['Area_Density']=CF_Area_Density.predict(cf)
df_cf['Tensile Strength']=CF_Tensile_Strength.predict(cf)
df_cf['Cost']=0.0551*df_cf['Area_Density']+43.89

df_cf

In [ ]:
df_new  = df_cf
df_new=df_new.drop(['Warp_Count',"Weft_Count","Warp_Filament_Count","Weft_Filament_Count","Warp_Filament_dia","Weft_Filament_dia","Weave","Type","Area_Density"],axis=1)

column = df_new.pop('Thickness')
df_new.insert(7, 'Thickness', column)
df_new=df_new.round(decimals = 4)

In [ ]:
df_new=df_new.sort_index()
df_new['Type'] = list_of_material
df_new

In [ ]:
df_new = pd.read_csv('properties.csv', names=('E1', 'E2', 'Poissons_ratio_12', 'Shear_Modulus_12','Shear_Modulus_13','Shear_Modulus_23','Tensile strength','Thickness','Cost','Type'))
df_new

In [ ]:
#Optimization.
from scipy.optimize import differential_evolution
from numpy.random import rand
from numpy import exp, sqrt, cos, e, pi, meshgrid, arange
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import pandas as pd
from composites import laminated_plate

from itertools import combinations_with_replacement
import itertools

In [ ]:
#input Requirments
from csv import reader
# open file in read mode
with open('Properties.csv', 'r' , encoding='utf-8-sig') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Get all rows of csv from csv_reader object as list of tuples
    Lamina_Properties = list(map(tuple,csv_reader))

res = []
for tup in Lamina_Properties:
    # list comprehension to check for each case
    temp = []
    for ele in tup:
        try:
            temp.append(float(ele))
        except ValueError:
            temp.append(ele)
    res.append(tuple(temp[0:9]))

dictOfFabrics = { i : res[i] for i in range(0, len(res) ) }
dictOfFabrics = dict(map(reversed, dictOfFabrics.items()))
#dictOfFabrics[tuple(core_property)] = -1
print(dictOfFabrics)

fabric=[i[0:7] for i in dictOfFabrics]
fabric


In [ ]:
#CASE 1.

In [ ]:
def DE(name):
    def objective(v):
        #saving floats to list
        a_list_floats = v
        
        #converting float to list of intgers
        a_list_int = (list(map(int,a_list_floats)))

        #splitting list in half to (Fabric_No & stacking angles)
        middle_index = len(a_list_int)//2
        Fabric_No = a_list_int[:middle_index]
        
        ########################-symmetry Protocol-#############################    
        
        reverse_Fabric_No = Fabric_No[:][::-1]
        #Fabric_No[-1:] = [-1]
        Fabric_No.extend(reverse_Fabric_No)

        ##########################################################
        stacking_angles = a_list_int[middle_index:]
        reverse_stacking_angles = stacking_angles[:][::-1]
        stacking_angles.extend(reverse_stacking_angles)

        #dictionary to replace intgers with fabric properties
        reversed_Fab_dict = {dictOfFabrics[k]:k for k in dictOfFabrics}

        #applying fabric properties dictionary
        Fabric_No = [reversed_Fab_dict[elem] for elem in Fabric_No]
        #print(Fabric_No)

        #making dictionary to replace intgers with stacking angles
        my_dict = {0: 0, 1: 45, 2: -45, 3: 90}
        #reversed_dict = {my_dict[k]:k for k in my_dict}

        #applying stacking angle dictionary
        stacking_angles = [my_dict[elem] for elem in stacking_angles]
        #print(stacking_angles)
        #=====================================================================
        # splitting Fab properties into elastic prop, Thickness, price--------

        Fabric_elastic_prop = [i[0:6] for i in Fabric_No]
        #Fabric_elastic_prop = Fabric_elastic_prop[findMiddle(Fabric_elastic_prop)] = core_property
        
        Fabric_strength =  [i[6] for i in Fabric_No]
            
        Fabric_thickness = [i[7] for i in Fabric_No]
        #Fabric_thickness = Fabric_thickness[findMiddle(Fabric_thickness)] = core_thickness
        
        Fabric_price = [i[8] for i in Fabric_No]
        
        #Fabric_price= Fabric_price[findMiddle(Fabric_price)] = 0
        #=====================================================================
        # total cost calc

        total_price = sum(Fabric_price)
        #print("Total cost = "+str(total_price))

        #print(Fabric_elastic_prop)
        #print(Fabric_thickness)
        #print(Fabric_price)

        #====================================================================
        #  CLT  ------------------------------------------------------------

        laminapropss = Fabric_elastic_prop

        #Thicknesses = Fabric_thickness
        #DF5 = pd.DataFrame(Thicknesses)
        #DF5.columns = ['Thicknesses']
        #print(DF5)
        #df_n = df_new[['Thickness']]
        #print(df_n)
        #bool_index = df_n['Thickness'].isin(DF5['Thicknesses'])
        #print(bool_index)
        #index_labels = df_n.index[bool_index]
        #print(index_labels)
        
        Thicknesses = Fabric_thickness
        DF5 = pd.DataFrame(Thicknesses)
        DF5.columns = ['Thicknesses']
        df_n = df_new[['Thickness']]
        index_labels = []
        for i, row in df_n.iterrows():
            if row['Thickness'] in Thicknesses:
                index_labels.append(i)
        print(type(index_labels))
        
        stack = stacking_angles

        plate = laminated_plate(stack, laminaprops = laminapropss, plyts = Thicknesses)
        
        Actual_thickness = sum(Fabric_thickness)

        plate.calc_equivalent_properties()
        
        A = plate.ABD
        A_mat =A[:3,:3]
        A_mat
        A_matInv=np.linalg.inv(A_mat)
        C = 0
        F = 0
        x = 'True'
        while x == 'True':
            C=C+1;
            F=F+.01;
            Force_mat=np.array([[F],[0],[0]])
            strain= np.dot(A_matInv,Force_mat)
            print(strain)
            Comp_mat = np.array([[1 / (df_new["E1"].iloc[index_labels]), 
             -df_new["Poissons_ratio_12"].iloc[index_labels] / df_new["E1"].iloc[index_labels], 
             0],
            [-df_new["Poissons_ratio_12"].iloc[index_labels] / df_new["E1"].iloc[index_labels],
            1/(df_new["E2"].iloc[index_labels]),
            0],
            [0,0,1/df_new["Shear_Modulus_12"].iloc[index_labels]]])
            Comp_matInv=np.linalg.inv(Comp_mat)
            Actual_stress= np.dot(Comp_matInv,strain)
            Actual_stress1 = Actual_stress[0][0]
            print(Actual_stress1)
            if df_new['Tensile strength'].iloc[index_labels] <= Actual_stress1:
                x = 'False'
                Tcomp = Actual_stress1
        #x--------------------x
        #x = 'True'
        #while x == 'True':
            #C=C+1;
            #F=F+.01;
            #Force_mat=np.array([[F],[0],[0]])
            #strain= np.dot(A_matInv,Force_mat)
            #print(strain)
            #Comp_mat = np.array([[1 / (df_new["E1"].iloc[index_labels]), 
                     -df_new["Poissons_ratio_12"].iloc[index_labels] / df_new["E1"].iloc[index_labels], 
                     0],
                    [-df_new["Poissons_ratio_12"].iloc[index_labels] / df_new["E1"].iloc[index_labels],
                     1/(df_new["E2"].iloc[index_labels]),
                     0],
    [0,0,1/df_new["Shear_Modulus_12"].iloc[index_labels]]])
            #Comp_matInv=np.linalg.inv(Comp_mat)
            #Actual_stress= np.dot(Comp_matInv,strain)
            #Actual_stress1 = Actual_stress[0][0]
            #print(Actual_stress1)
            
            #if df_new['Tensile strength'].iloc[index_labels] <= Actual_stress1:
                #x = 'False'
                #Tcomp = Actual_stress1
            #df = pd.DataFrame({'Tensile strength': [100, 200, 150, 250]})
            #Actual_stress1 = 175
            #i = [0, 2, 3]
            #result = df['Tensile strength'].iloc[index_labels] <= Actual_stress1
            #print(result)
        



        #laminate_height = plate.h
        #============================================================================
        #Max_cost= 100
        if (Actual_thickness > laminate_thickness ):
            y=999999999
        else:
            y = Actual_thickness
        #Max_cost= 100
        if (total_price > Max_cost ):
            x=100000
        else:
            x = -plate.D11
        
        if (x==100000 or y==999999999 ):
            xy=999999999
        else:
            xy = -plate.D11
        #print("Target Value = "+str(-x))
        #print("=================")
        return  xy
    #==========================================================

    laminas=int(name)

    bounds = [(0,len(Lamina_Properties))]*laminas+[(0,4)]*laminas
    # perform the differential evolution search
    result = differential_evolution(objective, bounds, workers=1)
    i=0
    i=i+1
    print(i)

    # summarize the result
    print('Status : %s' % result['message'])
    print('Total Evaluations: %d' % result['nfev'])
    # evaluate solution
    solution = result['x'].astype(int)
    print(solution)
    evaluation = objective(solution)
    print('\nSolution: f(%s) = %.5f' % (solution, evaluation))
    u=evaluation
    
    
    print("No of laminas are, " + str(int(name)+(int(name))))

    #all the below code is to print cost and target value
    
    middle_index = len(solution)//2
    Fabric_No = solution[:middle_index]
    Fabric1 = Fabric_No
    Fabric1.tolist()
    #Fabric1[-1:] = [-1]
    #saving fabric sequence in array for later use
    Fabric_sequence_array.append(Fabric1)
    
    Fabric_No=Fabric_No.tolist()
   
    #print((Fabric_No))
    
    
    reverse_Fabric_No = Fabric_No[:][::-1]
    #Fabric_No[-1:] = [-1]
    Fabric_No.extend(reverse_Fabric_No)
    
    #print("middle layer replaced with core below")
    print((Fabric_No))
    stacking_angles = solution[middle_index:]
    
    stacking_angle_temp = stacking_angles
    
    
    reverse_stacking_angles = stacking_angles[:][::-1]
    stacking_angles = np.append(stacking_angles, reverse_stacking_angles)

    #dictionary to replace intgers with fabric properties
    reversed_Fab_dict = {dictOfFabrics[k]:k for k in dictOfFabrics}

    #applying fabric properties dictionary
    Fabric_No = [reversed_Fab_dict[elem] for elem in Fabric_No]
    #print(Fabric_No)

    #making dictionary to replace intgers with stacking angles
    my_dict = {0: 0, 1: 45, 2: -45, 3: 90}
    #reversed_dict = {my_dict[k]:k for k in my_dict}

    #applying stacking angle dictionary
    stacking_angles = [my_dict[elem] for elem in stacking_angles]
    stacking_angle_converted = [my_dict[elem] for elem in stacking_angle_temp]
    stacking_angle_array.append(stacking_angle_converted)
    print(stacking_angles)
    
    
    #====================================================
    #print(Fabric_No)
    Fabric_elastic_prop = [i[0:6] for i in Fabric_No]
    #Fabric_elastic_prop = Fabric_elastic_prop[findMiddle(Fabric_elastic_prop)] = core_property
    
    Fabric_strength =  [i[6] for i in Fabric_No]
    
    Fabric_thickness = [i[7] for i in Fabric_No]
    #Fabric_thickness = Fabric_thickness[findMiddle(Fabric_thickness)] = core_thickness
    
    Fabric_price = [i[8] for i in Fabric_No]
    #Fabric_price= Fabric_price[findMiddle(Fabric_price)] = 0


    #=====================================================================
    # total cost calc

    total_price = sum(Fabric_price)
    print("Total cost = "+str(total_price))
    Total_Cost_array.append(total_price)
    D11_array.append(evaluation)
    Strength_array.append(Fabric_strength)
    #print(Fabric_elastic_prop)
    #print(Fabric_thickness)
    #print(Fabric_price)
    print("Fabric Strength = "+str(Fabric_strength))
    print("Laminate_thickness = "+str(sum(Fabric_thickness)))
    print("Target Value = "+str(evaluation))
    #print(str(evaluation))
    print("=================")
    return  str(evaluation)


In [ ]:
#Constraint
Max_cost= 650   # Dollar/sq meter
laminate_thickness =5

In [ ]:
global Total_Cost_array
global D11_array
global Strength_array
global Fabric_sequence_array
global stacking_angle_array
global stacking_angle_array1
Total_Cost_array = [0]
D11_array = [0]
Strength_array =[0]
Fabric_sequence_array = [0]
stacking_angle_array = [0]
stacking_angle_array1 = [0]
#for loop for evenly increasing lamina, runs untill a positive value is retured from DE final output
# so final result is the second last result
result_no=1
i = 2
k=-5
while k < 100:
    print("Solution NO = "+ str(result_no))
    string = (str(DE(str(i))))
    print("\n")
    if float(string) > 99.0: # basically loop stop when positive value is retured
        Total_Cost_array_n = Total_Cost_array[:-1]
        D11_array_n = D11_array[:-1]
        D11_array_n = [i * -1 for i in D11_array_n]
        Strength_array_n = Strength_array[:-1]
        Strength_array_n = [i * -1 for i in Strength_array_n]
        print(D11_array)
        print(Strength_array_n)
        print("NO FURTHER OPTIMIZATION POSSIBLE CONSIDER "+ str(D11_array_n.index(max(D11_array_n))) +"th SOLUTION \n \n")
        print("NO FURTHER OPTIMIZATION POSSIBLE CONSIDER "+ str(Strength_array_n.index(max(Strength_array_n))) +"th SOLUTION \n \n")
        print(df_new[["E1","E2","Thickness","Cost","Type"]])
        
        import numpy as np
        import matplotlib.pyplot as plt

        plt.title("graph")
        plt.xlabel("Cost")
        plt.ylabel("D11")
        plt.plot(Total_Cost_array_n, D11_array_n,"-o", label = "cost vs d11")
        plt.plot([Max_cost]*(len(D11_array_n)), D11_array_n, label = "max cost")
        plt.legend()
        plt.show()
        
        plt.title("graph")
        plt.xlabel("Cost")
        plt.ylabel("Strength")
        plt.plot(Total_Cost_array_n, Strength_array_n,"-o", label = "cost vs Strength")
        plt.plot([Max_cost]*(len(Strength_array_n)), Strength_array_n, label = "max cost")
        plt.legend()
        plt.show()
        
        material_properties = df_new[["E1","E2","Thickness","Cost","Type"]]
        material_properties['Fabrics No'] = material_properties.index
        material_properties
        first_column = material_properties.pop('Fabrics No')
        material_properties.insert(0, 'Fabrics No', first_column)
        #material_properties

        #core_c = core_property[:]
        #del core_c[2:6]
        #core_c.insert(0, -1)
        #core_c.insert(5, "Core")
        #to_append = core_c
        #a_series = pd.Series(to_append, index = material_properties.columns)
        #materials = material_properties.append(a_series, ignore_index=True)
        materials = material_properties
        print("Material Reference Table")
        #materials
        
        
        
        FN = pd.DataFrame(Fabric_sequence_array[D11_array_n.index(max(D11_array_n))], columns =['Fabric.NO'])
        reversed_Fab_dict = {dictOfFabrics[k]:k for k in dictOfFabrics}
        FN1 = pd.DataFrame(Fabric_sequence_array[Strength_array_n.index(max(Strength_array_n))], columns =['Fabric.NO'])
        reversed_Fab_dict = {dictOfFabrics[k]:k for k in dictOfFabrics}

        FF = Fabric_sequence_array[D11_array_n.index(max(D11_array_n))]
        FF = [reversed_Fab_dict[elem] for elem in FF]
        print(FF)
        FF1 = Fabric_sequence_array[Strength_array_n.index(max(Strength_array_n))]
        FF1 = [reversed_Fab_dict[elem] for elem in FF]
        print(FF1)

        Fabric_thickness = [i[7] for i in FF]
        Fabric_thickness
        FT = pd.DataFrame(Fabric_thickness, columns =['Thickness'])
        print(FT)
        fabricNo_to_material = dict (zip(materials['Fabrics No'], materials['Type']))
        Laminate_Layup = pd.concat([FN, FT], axis=1)
        Laminate_Layup["Material"] = Laminate_Layup["Fabric.NO"]
        #Laminate_Layup
        
        Fabric_thickness1 = [i[7] for i in FF1]
        Fabric_thickness1
        FT1 = pd.DataFrame(Fabric_thickness1, columns =['Thickness'])
        print(FT1)
        fabricNo_to_material1 = dict (zip(materials['Fabrics No'], materials['Type']))
        Laminate_Layup1 = pd.concat([FN1, FT1], axis=1)
        Laminate_Layup1["Material"] = Laminate_Layup1["Fabric.NO"]
        #Laminate_Layup

        Laminate_Layup=Laminate_Layup.replace({"Material": fabricNo_to_material})
        first_column = Laminate_Layup.pop('Material')
        Laminate_Layup.insert(0, 'Material', first_column)
        Laminate_Layup["Stacking Angle"] = (stacking_angle_array[D11_array_n.index(max(D11_array_n))])
        Laminate_Layup

        Laminate_Layup1=Laminate_Layup1.replace({"Material": fabricNo_to_material1})
        first_column1 = Laminate_Layup1.pop('Material')
        Laminate_Layup1.insert(0, 'Material', first_column1)
        Laminate_Layup1["Stacking Angle"] = (stacking_angle_array1[Strength_array_n.index(max(Strength_array_n))])
        Laminate_Layup1
        
        
        def color(val):
            if val == "Carbon":
                color = 'grey'
            elif val == "Glass":
                color = 'lightblue'
            elif val == "Core":
                color = 'yellow'
            return 'background-color: %s' % color
        
        
        
        break
    i += 1
    result_no += 1
 

In [ ]:
plate.ABD

In [ ]:
plt.title("graph")
plt.xlabel("Cost")
plt.ylabel("D11")
plt.plot(Total_Cost_array_n[0:int(ind)+1], D11_array_n[0:int(ind)+1],"-o", label = "cost vs D11")
plt.plot(Total_Cost_array_n[0:int(ind)+1],[minD11]*(int(ind)+1), label = "Min D11")
plt.legend()
plt.show()

In [ ]:
print("Material Reference Table")
materials

In [ ]:
print("Laminate Top Half Layup")
Laminate_Layup.style.applymap(color, subset=['Material']).set_properties(
    subset=['Material'], **{'width': '200px'}).set_properties(
    **{'border': '1px black solid !important'})